In [ ]:
import IJulia

In [ ]:
function rHCMVN(a1::AbstractArray{T, 1}, b1::AbstractArray{T, 1}, 
    Σ::Symmetric{T,Array{T,2}}, d::Int, m::Int;
    ns::Int = 10, N::Int = 1000, tol = convert(T, 1e-8),
    μ::Array{T,1} = zeros(T, length(a1))) where T<:AbstractFloat

    n = size(Σ, 1) # total size
    (n % m == 0) || throw(ArgumentError("The condition m|n must be met."))
    (n >= m) || throw(ArgumentError("The condition n >= m must be met."))

    a = copy(a1)
    b = copy(b1)
    a -= μ # centering
    b -= μ # centering
    
    if (n == m)
        p, y = CMVN(Σ, a, b, d, m = m, ns = ns, N = N, tol = tol)
        return p
    else
        B, UV = hchol(Σ, m)
        c = 0.03   ## beta, in paper
        reind = Blockreorder(n, c, [1:1:n;], a, b, m)
        B = B[reind, reind]
        nb = length(B) # the number of blocks, r in Cao2019
        x = zeros(T, n) # record expectations
        log_P = 0.0

        for i in 1:nb
            j = (i-1) * m
            if i>1
                i1 = UV[i-1].i1
                j1 = UV[i-1].j1
                bsz = UV[i-1].bsz
                delta = zeros(T, bsz) # g in Cao2019
                if UV[i-1].rank != 0
                    delta[1:bsz] .= UV[i-1].U * (transpose(UV[i-1].V) * x[j1:j1+bsz-1] )
                end
                a[i1:i1+bsz-1] -= delta 
                b[i1:i1+bsz-1] -= delta 
            end
            ai = a[j+1:j+m]
            bi = b[j+1:j+m]
            Σi = Symmetric(B[i] * B[i]')          
            pi, yi = CMVN(Σi, ai, bi, d, ns = ns, N = N, tol = tol)
            log_P += log(pi) # for numerical stability
            x[j+1:j+m] .= B[i] \ yi
        end

        return exp(log_P)
    end
    
end

function rHRCMVN(a1::AbstractArray{T, 1}, b1::AbstractArray{T, 1}, 
    Σ::Symmetric{T,Array{T,2}}, d::Int, m::Int;
    ns::Int = 10, N::Int = 1000, tol = convert(T, 1e-8),
    μ::Array{T,1} = zeros(T, length(a1))) where T<:AbstractFloat

    n = size(Σ, 1) # total size
    (n % m == 0) || throw(ArgumentError("The condition m|n must be met."))
    (n >= m) || throw(ArgumentError("The condition n >= m must be met."))

    a = copy(a1)
    b = copy(b1)
    a -= μ # centering
    b -= μ # centering
    
    if (n == m)
        p, y = RCMVN(Σ, a, b, d, m = m, ns = ns, N = N, tol = tol)
        return p
    else
        
        B, UV = hchol(Σ, m)
        c = 0.03   ## beta, in paper  
        reind = Blockreorder(n, c, [1:1:n;], a, b, m)   ## blockreorder
        B = B[reind, reind]
        nb = length(B) # the number of blocks, r in Cao2019
        x = zeros(T, n) # record expectations
        log_P = 0.0

        for i in 1:nb
            j = (i-1) * m
            if i>1
                i1 = UV[i-1].i1
                j1 = UV[i-1].j1
                bsz = UV[i-1].bsz
                delta = zeros(T, bsz) # g in Cao2019
                if UV[i-1].rank != 0
                    delta[1:bsz] .= UV[i-1].U * (transpose(UV[i-1].V) * x[j1:j1+bsz-1] )
                end
                a[i1:i1+bsz-1] -= delta 
                b[i1:i1+bsz-1] -= delta 
            end
            ai = a[j+1:j+m]
            bi = b[j+1:j+m]
            Σi = Symmetric(B[i] * B[i]')          
            pi, yi = RCMVN(Σi, ai, bi, d, ns = ns, N = N, tol = tol)
            log_P += log(pi) # for numerical stability
            x[j+1:j+m] .= B[i] \ yi
        end

        return exp(log_P)
    end
    
end